# Kmeans over a set of GeoTiffs

This notebook loads a set of GeoTiffs into a **RDD** of Tiles, with each Tile being a band in the GeoTiff. Each GeoTiff file contains [**MODIS MCD12Q2 v005**](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mcd12q2) for one year over the entire USA. The data ranges from 2001 until 2014.

Kmeans takes years as dimensions. Hence, the matrix has cells as rows and the years as columns. To cluster on all years, the matrix needs to be transposed. The notebook has two flavors of matrix transpose, locally by the Spark-driver or distributed using the Spark-workers. Once transposed the matrix is converted to a **RDD** of dense vectors to be used by **Kmeans** algorithm from **Spark-MLlib**. The end result is a grid where each cell has a cluster ID which is then saved into a SingleBand GeoTiff. By saving the result into a GeoTiff, the reader can plot it using a Python notebook as the one defined in the [python examples](../examples/python).

<span style="color:red">In this notebook the reader only needs to modify the variables in **Mode of Operation Setup**</span>.

## Dependencies

In [1]:
import sys.process._

import java.io.{ByteArrayInputStream, ByteArrayOutputStream, ObjectInputStream, ObjectOutputStream}


import geotrellis.proj4.CRS
import geotrellis.raster.{CellType, ArrayTile, DoubleArrayTile, Tile, UByteCellType}
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{GeoTiff, SinglebandGeoTiff}
import geotrellis.spark.io.hadoop._
import org.apache.hadoop.io._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.distributed.{CoordinateMatrix, MatrixEntry, RowMatrix}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.hadoop.io.{IOUtils, SequenceFile}
import org.apache.hadoop.io.SequenceFile.Writer

//Spire is a numeric library for Scala which is intended to be generic, fast, and precise.
import spire.syntax.cfor._

...


## Mode of operation

Here the user can define the mode of operation.
* **rdd_offline_mode**: If false it means the notebook will create all data from scratch and store protected_extent and num_cols_rows into HDFS. Otherwise, these data structures are read from HDFS.
* **matrix_offline_mode**: If false it means the notebook will create a mtrix,  transposed it and save it to HDFS. Otherwise, these data structures are read from HDFS.
* **kmeans_offline_mode**: If false it means the notebook will train kmeans and run kemans and store kmeans model into HDFS. Otherwise, these data structures are read from HDFS.

It is also possible to define which directory of GeoTiffs is to be used and on which **band** to run Kmeans. The options are
* **all** which are a multi-band (**8 bands**) GeoTiffs
* Or choose single band ones:
    0. Onset_Greenness_Increase
    1. Onset_Greenness_Maximum
    2. Onset_Greenness_Decrease
    3. Onset_Greenness_Minimum
    4. NBAR_EVI_Onset_Greenness_Minimum
    5. NBAR_EVI_Onset_Greenness_Maximum
    6. NBAR_EVI_Area
    7. Dynamics_QC

For kmeans the user can define the **number of iterations** and **number of clusters** as an inclusive range. Such range is defined using **minClusters**, **maxClusters**, and **stepClusters**. These variables will set a loop starting at **minClusters** and stopping at **maxClusters** (inclusive), iterating **stepClusters** at the time. <span style="color:red">Note that when using a range **kemans offline mode** is not possible and it will be reset to **online mode**</span>.

### Mode of Operation setup
<a id='mode_of_operation_setup'></a>

In [2]:
//Operation mode
var rdd_offline_mode = true
var matrix_offline_mode = true
var kmeans_offline_mode = true

//GeoTiffs to be read from "hdfs:///user/hadoop/modis/"
var dir_path = "hdfs:///user/hadoop/avhrr/"
var offline_dir_path = "hdfs:///user/emma/avhrr/"
/*
Choose all and then the band or the dir which has the band extracted.
0: Onset_Greenness_Increase
1: Onset_Greenness_Maximum
2: Onset_Greenness_Decrease
3: Onset_Greenness_Minimum
4: NBAR_EVI_Onset_Greenness_Minimum
5: NBAR_EVI_Onset_Greenness_Maximum
6: NBAR_EVI_Area
7: Dynamics_QC

for example:
var geoTiff_dir = "Onset_Greenness_Increase" 
var band_num = 0
*/
var geoTiff_dir = "SOST" 
var band_num = 0

//Years between (inclusive) 1989 - 2014
var satellite_first_year = 1989
var satellite_last_year = 2014

//Mask
val toBeMasked = true
val mask_path = "hdfs:///user/hadoop/usa_mask.tif"

//Kmeans number of iterations and clusters
var numIterations = 75
var minClusters = 100
var maxClusters = 120
var stepClusters = 10
var save_kmeans_model = false

rdd_offline_mode = true
matrix_offline_mode = true
kmeans_offline_mode = true
dir_path = hdfs:///user/hadoop/avhrr/
offline_dir_path = hdfs:///user/emma/avhrr/
geoTiff_dir = SOST
band_num = 0
satellite_first_year = 1989
satellite_last_year = 2014
toBeMasked = true
mask_path = hdfs:///user/hadoop/usa_mask.tif
numIterations = 75
minClusters = 100
maxClusters = 120
stepClusters = 10
save_kmeans_model = false


false


<span style="color:red">DON'T MODIFY ANY PIECE OF CODE FROM HERE ON!!!</span>.


### Mode of operation validation

In [3]:
//Validation, do not modify these lines.
var single_band = false
if (geoTiff_dir == "all") {
    single_band = false
} else {
    single_band = true
    if (band_num > 0) {
        println("Since it is single band, we will use band 0!!!")
        band_num  = 0
    }
}

if (minClusters > maxClusters) {
    maxClusters = minClusters
    stepClusters = 1
}
if (stepClusters < 1) {
    stepClusters = 1
}

//Paths to store data structures for Offline runs
var mask_str = ""
if (toBeMasked)
    mask_str = "_mask"
var grid0_path = offline_dir_path + geoTiff_dir + "/grid0" + "_"+ band_num + mask_str
var grid0_index_path = offline_dir_path + geoTiff_dir + "/grid0_index" + "_"+ band_num + mask_str
var grids_noNaN_path = offline_dir_path + geoTiff_dir + "/grids_noNaN" + "_"+ band_num + mask_str
var metadata_path = offline_dir_path + geoTiff_dir + "/metadata" + "_"+ band_num + mask_str
var grids_matrix_path = offline_dir_path + geoTiff_dir + "/grids_matrix" + "_"+ band_num + mask_str

//Check offline modes
var conf = sc.hadoopConfiguration
var fs = org.apache.hadoop.fs.FileSystem.get(conf)

val rdd_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grid0_path))
val matrix_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(grids_matrix_path))
                                      
if (rdd_offline_mode != rdd_offline_exists) {
    println("\"Load GeoTiffs\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + rdd_offline_exists.toString())
    rdd_offline_mode = rdd_offline_exists
} 
if (matrix_offline_mode != matrix_offline_exists) {
    println("\"Matrix\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + matrix_offline_exists.toString())
    matrix_offline_mode = matrix_offline_exists
}

if (!fs.exists(new org.apache.hadoop.fs.Path(mask_path))) {
    println("The mask path: " + mask_path + " is invalid!!!")
}

//Years
val satellite_years = 1980 to 2015

if (!satellite_years.contains(satellite_first_year) || !(satellite_years.contains(satellite_last_year))) {
  println("Invalid range of years for " + geoTiff_dir + ". I should be between " + satellite_first_year + " and " + satellite_last_year)
  System.exit(0)
}

var satellite_years_range = (satellite_years.indexOf(satellite_first_year), satellite_years.indexOf(satellite_last_year))

var num_kmeans :Int  = 1
if (minClusters != maxClusters) {
    num_kmeans = ((maxClusters - minClusters) / stepClusters) + 1
}
println(num_kmeans)
var kmeans_model_paths :Array[String] = Array.fill[String](num_kmeans)("")
var wssse_path :String = offline_dir_path + geoTiff_dir + "/75_wssse"
var geotiff_hdfs_paths :Array[String] = Array.fill[String](num_kmeans)("")
var geotiff_tmp_paths :Array[String] = Array.fill[String](num_kmeans)("")

if (num_kmeans > 1) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        kmeans_model_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + band_num + "_" + numClusters + "_" + numIterations
        
        //Check if the file exists
        val kmeans_exist = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))
        if (kmeans_exist && !kmeans_offline_mode) {
            println("The kmeans model path " + kmeans_model_paths(numClusters_id) + " exists, please remove it.")
        } else if (!kmeans_exist && kmeans_offline_mode) {
            kmeans_offline_mode = false
        }
        
        geotiff_hdfs_paths(numClusters_id) = offline_dir_path + geoTiff_dir + "/clusters_" + band_num + "_" + numClusters + "_" + numIterations + ".tif"
        geotiff_tmp_paths(numClusters_id) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + numClusters + "_" + numIterations + ".tif"
        if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(numClusters_id)))) {
            println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(numClusters_id) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
        }
        numClusters_id += 1
    }
    kmeans_offline_mode = false
} else { 
    kmeans_model_paths(0) = offline_dir_path + geoTiff_dir + "/kmeans_model_" + band_num + "_" + minClusters + "_" + numIterations
    val kmeans_offline_exists = fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(0)))
    if (kmeans_offline_mode != kmeans_offline_exists) {
        println("\"Kmeans\" offline mode is not set properly, i.e., either it was set to false and the required file does not exist or vice-versa. We will reset it to " + kmeans_offline_exists.toString())
        kmeans_offline_mode = kmeans_offline_exists
    }
    geotiff_hdfs_paths(0) = offline_dir_path + geoTiff_dir + "/clusters_" + band_num + "_" + minClusters + "_" + numIterations + ".tif"
    geotiff_tmp_paths(0) = "/tmp/clusters_" + band_num + "_" + geoTiff_dir + "_" + minClusters + "_" + numIterations + ".tif"
    if (fs.exists(new org.apache.hadoop.fs.Path(geotiff_hdfs_paths(0)))) {
        println("There is already a GeoTiff with the path: " + geotiff_hdfs_paths(0) + ". Please make either a copy or move it to another location, otherwise, it will be over-written.")
    }
}

Waiting for a Spark session to start...

3


single_band = true
mask_str = _mask
grid0_path = hdfs:///user/emma/avhrr/SOST/grid0_0_mask
grid0_index_path = hdfs:///user/emma/avhrr/SOST/grid0_index_0_mask
grids_noNaN_path = hdfs:///user/emma/avhrr/SOST/grids_noNaN_0_mask
metadata_path = hdfs:///user/emma/avhrr/SOST/metadata_0_mask
grids_matrix_path = hdfs:///user/emma/avhrr/SOST/grids_matrix_0_mask
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/usr/lib/spark-2.1.1-bin-without-hadoop/conf/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-1458383850_39, ugi=emma (auth:SIMPLE)]...


DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-1458383850_39, ugi=emma (auth:SIMPLE)]]

## Functions to (de)serialize any structure into Array[Byte]

In [4]:
def serialize(value: Any): Array[Byte] = {
    val out_stream: ByteArrayOutputStream = new ByteArrayOutputStream()
    val obj_out_stream = new ObjectOutputStream(out_stream)
    obj_out_stream.writeObject(value)
    obj_out_stream.close
    out_stream.toByteArray
}

def deserialize(bytes: Array[Byte]): Any = {
    val obj_in_stream = new ObjectInputStream(new ByteArrayInputStream(bytes))
    val value = obj_in_stream.readObject
    obj_in_stream.close
    value
}

serialize: (value: Any)Array[Byte]
deserialize: (bytes: Array[Byte])Any


## Load GeoTiffs

Using GeoTrellis all GeoTiffs of a directory will be loaded into a RDD. Using the RDD, we extract a grid from the first file to lated store the Kmeans cluster_IDS, we build an Index for populate such grid and we filter out here all NaN values.

In [5]:
val t0 = System.nanoTime()
//Global variables
var projected_extent = new ProjectedExtent(new Extent(0,0,0,0), CRS.fromName("EPSG:3857"))
var grid0: RDD[(Long, Double)] = sc.emptyRDD
var grid0_index: RDD[Long] = sc.emptyRDD
var grids_noNaN_RDD: RDD[Array[Double]] = sc.emptyRDD
var num_cols_rows :(Int, Int) = (0, 0)
var cellT :CellType = UByteCellType
var grids_RDD :RDD[Array[Double]] = sc.emptyRDD
var mask_tile0 :Tile = new SinglebandGeoTiff(geotrellis.raster.ArrayTile.empty(cellT, num_cols_rows._1, num_cols_rows._2), projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions.DEFAULT).tile
var grid_cells_size :Long = 0

//Load Mask
if (toBeMasked) {
    val mask_tiles_RDD = sc.hadoopGeoTiffRDD(mask_path).values
    val mask_tiles_withIndex = mask_tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
    mask_tile0 = (mask_tiles_withIndex.filter(m => m._1==0).values.collect())(0)
}

//Local variables
val pattern: String = "tif"
val filepath: String = dir_path + geoTiff_dir

if (rdd_offline_mode) {
    grids_noNaN_RDD = sc.objectFile(grids_noNaN_path)
    grid0 = sc.objectFile(grid0_path)
    grid0_index = sc.objectFile(grid0_index_path)

    val metadata = sc.sequenceFile(metadata_path, classOf[IntWritable], classOf[BytesWritable]).map(_._2.copyBytes()).collect()
    projected_extent = deserialize(metadata(0)).asInstanceOf[ProjectedExtent]
    num_cols_rows = (deserialize(metadata(1)).asInstanceOf[Int], deserialize(metadata(2)).asInstanceOf[Int])
    cellT = deserialize(metadata(3)).asInstanceOf[CellType]  
} else {
    if (single_band) {
        //Lets load a Singleband GeoTiffs and return RDD just with the tiles.
        var tiles_RDD :RDD[Tile] = sc.hadoopGeoTiffRDD(filepath, pattern).values
            
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_RDD = tiles_RDD.map(m => m.localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
        } else {
            grids_RDD = tiles_RDD.map(m => m.toArrayDouble())
        }
    } else {
        //Lets load Multiband GeoTiffs and return RDD just with the tiles.
        val tiles_RDD = sc.hadoopMultibandGeoTiffRDD(filepath, pattern).values
    
        //Retrive the numbre of cols and rows of the Tile's grid
        val tiles_withIndex = tiles_RDD.zipWithIndex().map{case (e,v) => (v,e)}
        val tile0 = (tiles_withIndex.filter(m => m._1==0).values.collect())(0)
        num_cols_rows = (tile0.cols,tile0.rows)
        cellT = tile0.cellType
    
        //Lets read the average of the Spring-Index which is stored in the 4th band
        val band_numB :Broadcast[Int] = sc.broadcast(band_num)
        if (toBeMasked) {
            val mask_tile_broad :Broadcast[Tile] = sc.broadcast(mask_tile0)
            grids_RDD = tiles_RDD.map(m => m.band(band_numB.value).localInverseMask(mask_tile_broad.value, 1, -1000).toArrayDouble())
        } else {
            grids_RDD = tiles_RDD.map(m => m.band(band_numB.value).toArrayDouble())
        }
    }

    //Retrieve the ProjectExtent which contains metadata such as CRS and bounding box
    val projected_extents_withIndex = sc.hadoopGeoTiffRDD(filepath, pattern).keys.zipWithIndex().map{case (e,v) => (v,e)}
    projected_extent = (projected_extents_withIndex.filter(m => m._1 == 0).values.collect())(0)

    //Get Index for each Cell
    val grids_withIndex = grids_RDD.zipWithIndex().map { case (e, v) => (v, e) }
    if (toBeMasked) {
        grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.filter(m => m._1 != -1000.0).map { case (v, i) => (i) }
    } else {
        grid0_index = grids_withIndex.filter(m => m._1 == 0).values.flatMap(m => m).zipWithIndex.map { case (v, i) => (i) }

    }
    //Get the Tile's grid
    grid0 = grids_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}

    //Lets filter out NaN
    if (toBeMasked) {
        grids_noNaN_RDD = grids_RDD.map(m => m.filter(m => m != -1000.0))
    } else {
        grids_noNaN_RDD = grids_RDD
    }
    //Store data in HDFS
    grid0.saveAsObjectFile(grid0_path)
    grid0_index.saveAsObjectFile(grid0_index_path)
    grids_noNaN_RDD.saveAsObjectFile(grids_noNaN_path)
    
    val grids_noNaN_RDD_withIndex = grids_noNaN_RDD.zipWithIndex().map { case (e, v) => (v, e) }
    grids_noNaN_RDD = grids_noNaN_RDD_withIndex.filterByRange(satellite_years_range._1, satellite_years_range._2).values
    
    val writer: SequenceFile.Writer = SequenceFile.createWriter(conf,
        Writer.file(metadata_path),
        Writer.keyClass(classOf[IntWritable]),
        Writer.valueClass(classOf[BytesWritable])
    )

    writer.append(new IntWritable(1), new BytesWritable(serialize(projected_extent)))
    writer.append(new IntWritable(2), new BytesWritable(serialize(num_cols_rows._1)))
    writer.append(new IntWritable(3), new BytesWritable(serialize(num_cols_rows._2)))
    writer.append(new IntWritable(4), new BytesWritable(serialize(cellT)))
    writer.hflush()
    writer.close()
}
grid_cells_size = grid0_index.count().toInt
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 13843484890ns                                                     


t0 = 183711253169368
projected_extent = ProjectedExtent(Extent(-126.30312894720473, 14.29219617034159, -56.162671563152486, 49.25462702827337),geotrellis.proj4.CRS$$anon$3@41d0d1b7)
grid0 = MapPartitionsRDD[14] at objectFile at <console>:104
grid0_index = MapPartitionsRDD[16] at objectFile at <console>:105
grids_noNaN_RDD = MapPartitionsRDD[12] at objectFile at <console>:103
num_cols_rows = (7808,3892)
cellT = int16ud-1000
grids_RDD = EmptyRDD[3] at emptyRDD at <console>:87
mask_tile0 = FloatRawArrayTile([F@65ab79c1,7808,3892)


gri...


FloatRawArrayTile([F@65ab79c1,7808,3892)

In [6]:
grid0_index.count()

[Stage 3:======================================================>  (26 + 1) / 27]

10115631

In [7]:
val res = grids_noNaN_RDD.map(m => m.size)

res = MapPartitionsRDD[19] at map at <console>:58


MapPartitionsRDD[19] at map at <console>:58

In [8]:
res.collect()

[Stage 4:======================================================>  (25 + 1) / 26]

[10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631, 10115631]

## Matrix

We need to do a Matrix transpose to have clusters per cell and not per year. With a GeoTiff representing a single year, the loaded data looks liks this:
```
bands_RDD.map(s => Vectors.dense(s)).cache()

//The vectors are rows and therefore the matrix will look like this:
[
Vectors.dense(0.0, 1.0, 2.0),
Vectors.dense(3.0, 4.0, 5.0),
Vectors.dense(6.0, 7.0, 8.0),
Vectors.dense(9.0, 0.0, 1.0)
]
```

To achieve that we convert the **RDD[Vector]** into a distributed Matrix, a [**CoordinateMatrix**](https://spark.apache.org/docs/latest/mllib-data-types.html#coordinatematrix), which as a **transpose** method.

In [9]:
val t0 = System.nanoTime()
//Global variables
var grids_matrix: RDD[Vector] = sc.emptyRDD
val grid_cells_sizeB = sc.broadcast(grid_cells_size)

if (matrix_offline_mode) {
    grids_matrix = sc.objectFile(grids_matrix_path)
} else {
    val mat :RowMatrix = new RowMatrix(grids_noNaN_RDD.map(m => m.zipWithIndex).map(m => m.filter(!_._1.isNaN)).map(m => Vectors.sparse(grid_cells_sizeB.value.toInt, m.map(v => v._2), m.map(v => v._1))))
    // Split the matrix into one number per line.
    val byColumnAndRow = mat.rows.zipWithIndex.map {
        case (row, rowIndex) => row.toArray.zipWithIndex.map {
            case (number, columnIndex) => new MatrixEntry(rowIndex, columnIndex, number)
        }   
    }.flatMap(x => x)
    
    val matt: CoordinateMatrix = new CoordinateMatrix(byColumnAndRow)
    val matt_T = matt.transpose()
    //grids_matrix = matt_T.toRowMatrix().rows
    grids_matrix = matt_T.toIndexedRowMatrix().rows.sortBy(_.index).map(_.vector)
    grids_matrix.saveAsObjectFile(grids_matrix_path)
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 35167280ns


t0 = 183735830480327
grids_matrix = MapPartitionsRDD[22] at objectFile at <console>:75
grid_cells_sizeB = Broadcast(10)
t1 = 183735865647607


183735865647607

## Kmeans

We use Kmeans from Sparl-MLlib. The user should only modify the variables on Kmeans setup.

### Kmeans Training

In [10]:
val t0 = System.nanoTime()
//Global variables
var kmeans_models :Array[KMeansModel] = new Array[KMeansModel](num_kmeans)
var wssse_data :List[(Int, Int, Double)] = List.empty

if (kmeans_offline_mode) {
    var numClusters_id = 0
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
            println("One of the files does not exist, we will abort!!!")
            System.exit(0)
        } else {
            kmeans_models(numClusters_id) = KMeansModel.load(sc, kmeans_model_paths(numClusters_id))
        }
        numClusters_id += 1
    }
    val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
    wssse_data  = wssse_data_RDD.collect().toList
} else {
    var numClusters_id = 0
    if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
        val wssse_data_RDD :RDD[(Int, Int, Double)]  = sc.objectFile(wssse_path)
        wssse_data  = wssse_data_RDD.collect().toList
    }
    grids_matrix.cache()
    cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
        println(numClusters)
        kmeans_models(numClusters_id) = {
            KMeans.train(grids_matrix, numClusters, numIterations)
        }

        // Evaluate clustering by computing Within Set Sum of Squared Errors
        val WSSSE = kmeans_models(numClusters_id).computeCost(grids_matrix)
        println("Within Set Sum of Squared Errors = " + WSSSE)
                
        wssse_data = wssse_data :+ (numClusters, numIterations, WSSSE)
        
        //Save kmeans model
        if (save_kmeans_model) {
            if (!fs.exists(new org.apache.hadoop.fs.Path(kmeans_model_paths(numClusters_id)))) {
                kmeans_models(numClusters_id).save(sc, kmeans_model_paths(numClusters_id))
            }
        }
        numClusters_id += 1
        
        if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
            println("We will delete the wssse file")
            try { fs.delete(new org.apache.hadoop.fs.Path(wssse_path), true) } catch { case _ : Throwable => { } }
        }
    
        println("Lets create it with the new data")
        sc.parallelize(wssse_data, 1).saveAsObjectFile(wssse_path)
    }

    //Un-persist it to save memory
    grids_matrix.unpersist()
    

}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

100
Within Set Sum of Squared Errors = 2.3038760590606104E11                        
We will delete the wssse file
Lets create it with the new data
110
Within Set Sum of Squared Errors = 2.2701004021130945E11                        
We will delete the wssse file
Lets create it with the new data
120
Within Set Sum of Squared Errors = 2.2270836011686545E11                        
We will delete the wssse file
Lets create it with the new data
Elapsed time: 1141136943374ns


t0 = 183737586335476
kmeans_models = Array(org.apache.spark.mllib.clustering.KMeansModel@22a84dcf, org.apache.spark.mllib.clustering.KMeansModel@78463207, org.apache.spark.mllib.clustering.KMeansModel@5933f53e)
wssse_data = List((10,75,3.543594088100406E11), (20,75,3.1164671707882837E11), (30,75,2.891401201066724E11), (40,75,2.7659957276233606E11), (50,75,2.6393510342638516E11), (60,75,2.5526775470250888E11), (70,75,2.474527242908672E11), (80,75,2.4158928746856482E11), (90,75,2.3546354225689584E11), (100,75,2.3163178150354764E11), (110,75,2.2620214478763037E11), (120,75,2.231121099726902E11), (130,75,2.1968520531267715E11), (140,75,2.1637970796204584E11), (150,75,2.1311835641170624E11), (160,75,2.10099...


List((10,75,3.543594088100406E11), (20,75,3.1164671707882837E11), (30,75,2.891401201066724E11), (40,75,2.7659957276233606E11), (50,75,2.6393510342638516E11), (60,75,2.5526775470250888E11), (70,75,2.474527242908672E11), (80,75,2.4158928746856482E11), (90,75,2.3546354225689584E11), (100,75,2.3163178150354764E11), (110,75,2.2620214478763037E11), (120,75,2.231121099726902E11), (130,75,2.1968520531267715E11), (140,75,2.1637970796204584E11), (150,75,2.1311835641170624E11), (160,75,2.100999719988994E11), (170,75,2.0805760370421555E11), (180,75,2.0606506045153506E11), (190,75,2.034304514480542E11), (200,75,2.0235881533249454E11), (210,75,1.9951537144941003E11), (220,75,1.9824697995193225E11), (230,75,1.9573530305860495E11), (240,75,1.94558876227357E11), (250,75,1.925478193220847E11), (260,75,1.908033219040186E11), (270,75,1.8954723146873877E11), (280,75,1.8836500863517624E11), (290,75,1.8699108636352213E11), (300,75,1.858526017795722E11), (310,75,1.8423297708310217E11), (320,75,1.8317663056064

### Inspect WSSSE

In [11]:
val t0 = System.nanoTime()
//current
println(wssse_data)

//from disk
if (fs.exists(new org.apache.hadoop.fs.Path(wssse_path))) {
    var wssse_data_tmp :RDD[(Int, Int, Double)] = sc.objectFile(wssse_path)//.collect()//.toList
    println(wssse_data_tmp.collect().toList)    
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

List((10,75,3.543594088100406E11), (20,75,3.1164671707882837E11), (30,75,2.891401201066724E11), (40,75,2.7659957276233606E11), (50,75,2.6393510342638516E11), (60,75,2.5526775470250888E11), (70,75,2.474527242908672E11), (80,75,2.4158928746856482E11), (90,75,2.3546354225689584E11), (100,75,2.3163178150354764E11), (110,75,2.2620214478763037E11), (120,75,2.231121099726902E11), (130,75,2.1968520531267715E11), (140,75,2.1637970796204584E11), (150,75,2.1311835641170624E11), (160,75,2.100999719988994E11), (170,75,2.0805760370421555E11), (180,75,2.0606506045153506E11), (190,75,2.034304514480542E11), (200,75,2.0235881533249454E11), (210,75,1.9951537144941003E11), (220,75,1.9824697995193225E11), (230,75,1.9573530305860495E11), (240,75,1.94558876227357E11), (250,75,1.925478193220847E11), (260,75,1.908033219040186E11), (270,75,1.8954723146873877E11), (280,75,1.8836500863517624E11), (290,75,1.8699108636352213E11), (300,75,1.858526017795722E11), (310,75,1.8423297708310217E11), (320,75,1.8317663056064

t0 = 184880142940509
t1 = 184880305821358


184880305821358

### Run Kmeans clustering

Run Kmeans and obtain the clusters per each cell.

In [12]:
val t0 = System.nanoTime()
//Cache it so kmeans is more efficient
grids_matrix.cache()

var kmeans_res: Array[RDD[Int]] = Array.fill(num_kmeans)(sc.emptyRDD)
var numClusters_id = 0
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    kmeans_res(numClusters_id) = kmeans_models(numClusters_id).predict(grids_matrix)
    numClusters_id += 1
}

//Un-persist it to save memory
grids_matrix.unpersist()
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

Elapsed time: 25206308ns


t0 = 184881029051369
kmeans_res = Array(MapPartitionsRDD[543] at map at KMeansModel.scala:69, MapPartitionsRDD[544] at map at KMeansModel.scala:69, MapPartitionsRDD[545] at map at KMeansModel.scala:69)
numClusters_id = 3
t1 = 184881054257677


184881054257677

#### Sanity test

It can be skipped, it only shows the cluster ID for the first 50 cells

In [13]:
val t0 = System.nanoTime()
val kmeans_res_out = kmeans_res(0).filter(_!= 0).filter(_!=1).take(150)
kmeans_res_out.foreach(print)

println(kmeans_res_out.size)
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

353535353535353510101035351010103535353535353535875454545454545454545454545410353535353535351010101010103535353535353535353587545454545454545454545454543535353535353535353535353535353535353535353535358754545454545454545454545454353535353535353535353535353535353535353535353535355454545454545454545454150
Elapsed time: 154061853ns


t0 = 184882221864829
kmeans_res_out = Array(35, 35, 35, 35, 35, 35, 35, 35, 10, 10, 10, 35, 35, 10, 10, 10, 35, 35, 35, 35, 35, 35, 35, 35, 87, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 10, 35, 35, 35, 35, 35, 35, 35, 10, 10, 10, 10, 10, 10, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 87, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 87, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54)
t1 = 184882375926682


184882375926682

## Build GeoTiff with Kmeans cluster_IDs

The Grid with the cluster IDs is stored in a SingleBand GeoTiff and uploaded to HDFS.

### Assign cluster ID to each grid cell and save the grid as SingleBand GeoTiff

To assign the clusterID to each grid cell it is necessary to get the indices of gird cells they belong to. The process is not straight forward because the ArrayDouble used for the creation of each dense Vector does not contain the NaN values, therefore there is not a direct between the indices in the Tile's grid and the ones in **kmeans_res** (kmeans result).

To join the two RDDS the knowledge was obtaing from a stackoverflow post on [how to perform basic joins of two rdd tables in spark using python](https://stackoverflow.com/questions/31257077/how-do-you-perform-basic-joins-of-two-rdd-tables-in-spark-using-python).

In [14]:
val t0 = System.nanoTime()
var numClusters_id = 0
val grid0_index_I = grid0_index.zipWithIndex().map{ case (v,i) => (i,v)}
cfor(minClusters)(_ <= maxClusters, _ + stepClusters) { numClusters =>
    //Merge two RDDs, one containing the clusters_ID indices and the other one the indices of a Tile's grid cells
    val cluster_cell_pos = ((kmeans_res(numClusters_id).zipWithIndex().map{ case (v,i) => (i,v)}).join(grid0_index_I).map{ case (k,(v,i)) => (v,i)})

    //Associate a Cluster_IDs to respective Grid_cell
    val grid_clusters = grid0.map{ case (i, v) => if (v == 0.0) (i,Double.NaN) else (i,v)}.leftOuterJoin(cluster_cell_pos.map{ case (c,i) => (i.toLong, c)})

    //Convert all None to NaN
    val grid_clusters_res = grid_clusters.sortByKey(true).map{case (k, (v, c)) => if (c == None) (k, Double.NaN) else (k, c.get.toDouble)}
    
    //Define a Tile
    val cluster_cells :Array[Double] = grid_clusters_res.values.collect()
    val cluster_cellsD = DoubleArrayTile(cluster_cells, num_cols_rows._1, num_cols_rows._2)
    //val cluster_tile = geotrellis.raster.DoubleArrayTile.empty(num_cols_rows._1, num_cols_rows._2)
    //cfor(0)(_ < num_cols_rows._1, _ + 1) { col =>
    //    cfor(0)(_ < num_cols_rows._2, _ + 1) { row =>
    //        val v = cluster_cellsD.getDouble(col, row)
    //        if (v != Double.NaN)
    //            cluster_tile.setDouble(col, row, v)
    //    }
    //}

    //val geoTif = new SinglebandGeoTiff(cluster_tile, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    val geoTif = new SinglebandGeoTiff(cluster_cellsD, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    //val corr_tile_projec = cluster_tile.reproject(projected_extent.extent, projected_extent.crs, CRS.fromName("EPSG:4326"))
    //val geoTif = new SinglebandGeoTiff(corr_tile_projec, corr_tile_projec.extent, CRS.fromName("EPSG:4326"), Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    
    //Save to /tmp/
    GeoTiffWriter.write(geoTif, geotiff_tmp_paths(numClusters_id))

    //Upload to HDFS
    var cmd = "hadoop dfs -copyFromLocal -f " + geotiff_tmp_paths(numClusters_id) + " " + geotiff_hdfs_paths(numClusters_id)
    println(cmd)
    Process(cmd)!
    
    //Remove from /tmp/
    cmd = "rm -fr " + geotiff_tmp_paths(numClusters_id)
    println(cmd)
    Process(cmd)!
    
    numClusters_id += 1
}
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0) + "ns")

[Stage 500:================================================>      (28 + 4) / 32]hadoop dfs -copyFromLocal -f /tmp/clusters_0_SOST_100_75.tif hdfs:///user/emma/avhrr/SOST/clusters_0_100_75.tif
DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

rm -fr /tmp/clusters_0_SOST_100_75.tif
[Stage 512:=====================================================> (31 + 1) / 32]hadoop dfs -copyFromLocal -f /tmp/clusters_0_SOST_110_75.tif hdfs:///user/emma/avhrr/SOST/clusters_0_110_75.tif
DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

rm -fr /tmp/clusters_0_SOST_110_75.tif
[Stage 524:===================================================>   (30 + 2) / 32]hadoop dfs -copyFromLocal -f /tmp/clusters_0_SOST_120_75.tif hdfs:///user/emma/avhrr/SOST/clusters_0_120_75.tif
DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

rm -fr /tmp/clusters_0

t0 = 184883632334949
numClusters_id = 3
grid0_index_I = MapPartitionsRDD[549] at map at <console>:77
t1 = 185163756710949


185163756710949

# [Visualize results](plot_kmeans_clusters.ipynb) --------------- [Plot WSSE](kmeans_wsse.ipynb)